# KNN Time Series Forecasting for NVIDIA Stock Prices

## Data Extraction

In [ ]:
pip install yfinance

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

import yfinance as yf

In [2]:
# Define the stock list
stocks_list = ['NVDA']

# Define the date range
end = datetime.now()
start = datetime(end.year - 2, end.month, end.day)

# Download stock data
for stock in stocks_list:
    globals()[stock] = yf.download(stock, start, end)

# Reset the index and rename columns
NVDA.reset_index(inplace=True)
NVDA.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

# Check for NaN values in each column
print("\nNumber of NaN values in each column:")
print(NVDA.isna().sum())

[*********************100%***********************]  1 of 1 completed


Number of NaN values in each column:
Date         0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64


## Data Pre-Processing

In [3]:
# Create a new data frame with only the 'Close' column
nvidia_data = NVDA[['Close']]

# Drop rows where 'Close' is NaN
nvidia_data = nvidia_data.dropna()

# Print the resulting DataFrame after dropping NaN values
print("\nDataFrame after dropping NaN values:")
print(nvidia_data.head())

# Check if the resulting DataFrame is empty
if nvidia_data.empty:
    raise ValueError("All values in the 'Close' column are NaN. Please check your data source.")

# Convert the DataFrame to a NumPy array
data = nvidia_data.values

# Print the resulting data array
print("\nResulting data array:")
print(data)


DataFrame after dropping NaN values:
    Close
0  13.497
1  13.543
2  13.219
3  13.421
4  14.156

Resulting data array:
[[ 13.49699974]
 [ 13.54300022]
 [ 13.21899986]
 [ 13.42099953]
 [ 14.15600014]
 [ 14.30099964]
 [ 14.60200024]
 [ 13.77600002]
 [ 15.75      ]
 [ 16.32699966]
 [ 16.29500008]
 [ 16.66600037]
 [ 15.90999985]
 [ 15.67700005]
 [ 15.4090004 ]
 [ 15.31700039]
 [ 16.03800011]
 [ 16.5189991 ]
 [ 16.27000046]
 [ 15.82699966]
 [ 15.63899994]
 [ 16.92300034]
 [ 17.13500023]
 [ 16.87599945]
 [ 16.61000061]
 [ 15.98700047]
 [ 16.12000084]
 [ 17.16900063]
 [ 17.00099945]
 [ 17.53499985]
 [ 18.0720005 ]
 [ 17.67399979]
 [ 16.95199966]
 [ 16.57099915]
 [ 16.25399971]
 [ 16.08499908]
 [ 16.50099945]
 [ 15.33899975]
 [ 15.20600033]
 [ 14.12100029]
 [ 14.03600025]
 [ 14.60299969]
 [ 14.61400032]
 [ 14.31499958]
 [ 14.7489996 ]
 [ 14.26500034]
 [ 14.85900021]
 [ 15.62800026]
 [ 15.9090004 ]
 [ 16.00099945]
 [ 16.51099968]
 [ 16.89900017]
 [ 17.70199966]
 [ 17.37700081]
 [ 16.76499939]

In [4]:
# Standardize the data (standardization is less sensitive to outliers compared to normalization)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

In [5]:
# Split the scaled data into training and testing sets
training_data_len = int(np.ceil(len(scaled_data) * .8))
train_data = scaled_data[0:int(training_data_len), :]

# Split the data into x_train, y_train datasets
x_train, y_train = [], []
for i in range(60, len(train_data)):
    x_train.append(train_data[i - 60:i, 0])
    y_train.append(train_data[i,0])

# Convert the x_train and y_train to numppy array and reshape the data
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(341, 60, 1)

## K-Nearest Neighbors (KNN)

In [10]:
# Build Model and Fit
knn_model = KNeighborsClassifier(n_neighbors = 5) # Using 5 neighbors, could use 1?
knn_model.fit(x_train, y_train)
# Get prediction using x_test
knn_y_predict = knn_model.predict(x_test)
# Convert to Scaler
knn_y_predict = scaler.inverse_transform(knn_y_predict)



ValueError: Found array with dim 3. KNeighborsClassifier expected <= 2.

In [ ]:
# Evaluate KNN Model using RMSE
knn_rmse = np.sqrt(mean_squared_error(y_test, knn_y_predict))
print(f'Root Mean Squared Error: {knn_rmse}')

In [ ]:
# Plot the predicted stock prices against KNN actual prices
dataset = nvidia_data.filter(['Close'])
train = dataset[:training_data_len]
test = dataset[training_data_len:]
test['Predictions'] = predictions

# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(test[['Close', 'Predictions']])
plt.legend(['Train', 'Test', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# Evaluate how well the KNN model performs.
# If the predictions perfectly match the actual values, the points should align along a straight line with a slope of 1.

actual_values = y_test
knn_predicted_values = knn_y_predict

plt.figure(figsize=(10, 6))

# Scatter plot of predicted vs. actual values
plt.subplot(2, 2, 1)
plt.scatter(knn_predicted_values, actual_values, alpha=0.7)
plt.title('Predicted vs. Actual Values')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')